In [1]:
import sys
import os

# Ruta al directorio raíz del proyecto (ajústala si es necesario)
ruta_raiz = os.path.abspath(r"C:\Users\Germán Llorente\Desktop\germiprogramer\TFG-Agente-de-Scouting")

if ruta_raiz not in sys.path:
    sys.path.append(ruta_raiz)

In [2]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [3]:
# Cargar los datasets
jugadores_laliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_laliga.csv')
jugadores_premier = pd.read_csv('datos/datos_jugadores_v3/jugadores_premier.csv')
jugadores_seriea = pd.read_csv('datos/datos_jugadores_v3/jugadores_seriea.csv')
jugadores_bundesliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_bundesliga.csv')

In [4]:
# Unir todos los datasets (uno debajo del otro)
jugadores_total = pd.concat([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga], ignore_index=True)
jugadores_total

,player_id,player_name,team,team_id,competition,tackle_success_rate,tackles_successful,interception_success_rate,interceptions,clearances,...,height_cm,weight_kg,club_jersey_number,club_loaned_from,club_contract_valid_until,nationality_id,nationality_name,preferred_foot,release_clause_eur,posicion_mas_jugada
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,Spain - La Liga,56.86,29,62.75,32,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3063,Danilo Luiz da Silva,Real Madrid,220,Spain - La Liga,64.18,43,61.11,33,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3084,Christian Atsu Twasam,Málaga,223,Spain - La Liga,66.67,4,66.67,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3130,Gaël Kakuta,Sevilla,213,Spain - La Liga,100.00,2,0.00,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3160,Rene Krhin,Granada,1049,Spain - La Liga,64.71,22,71.05,27,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,400621,Tohouri Zahoui Constant Djakpa,Eintracht Frankfurt,184,Germany - 1. Bundesliga,91.67,11,69.23,9,26,...,177.0,74.0,15.0,NaN,2016.0,108.0,Côte d'Ivoire,Left,NaN,Left Back
1957,400622,Joel Gerezgiher,Eintracht Frankfurt,184,Germany - 1. Bundesliga,100.00,1,66.67,2,1,...,176.0,71.0,32.0,NaN,2018.0,21.0,Germany,Right,NaN,Left Midfield
1958,400627,Dario Kresic,Bayer Leverkusen,904,Germany - 1. Bundesliga,0.00,0,0.00,0,0,...,196.0,96.0,25.0,NaN,2016.0,10.0,Croatia,Right,NaN,Goalkeeper
1959,400654,Roel Brouwers,Borussia Mönchengladbach,185,Germany - 1. Bundesliga,100.00,2,50.00,1,22,...,192.0,82.0,4.0,NaN,2016.0,34.0,Netherlands,Right,NaN,Right Center Back


In [9]:
import pandas as pd

# Lista de (player_id, team) válidos
ids_y_equipos_validos = [
    (3083, "Tottenham Hotspur"),
    (3089, "Manchester City"),
    (3302, "Watford"),
    (5474, "Inter Milan"),
    (5687, "Eibar"),
    (6593, "Real Betis"),
    (6743, "Rayo Vallecano"),
    (6950, "Swansea City"),
    (7141, "Levante UD"),
    (7170, "Carpi"),
    (7788, "Torino"),
    (7852, "Genoa"),
    (8217, "Juventus"),
    (9461, "Norwich City"),
    (10881, "Sevilla"),
    (17520, "VfB Stuttgart"),
    (18404, "Rayo Vallecano"),
    (23488, "Watford"),
    (26211, "Levante UD"),
    (27341, "Celta Vigo"),
    (27853, "Fiorentina"),
]

# Convertir a DataFrame
df_validos = pd.DataFrame(ids_y_equipos_validos, columns=['player_id', 'team'])

# Filtrar duplicados por player_id
duplicados = jugadores_total[jugadores_total.duplicated(subset='player_id', keep=False)]

# Mantener solo los duplicados correctos (player_id + team que coincidan con la lista)
duplicados_validos = duplicados.merge(df_validos, on=['player_id', 'team'])

# Mantener los jugadores únicos que no están duplicados
no_duplicados = jugadores_total[~jugadores_total['player_id'].isin(duplicados['player_id'])]

# Unir los válidos con los no duplicados
jugadores_total_filtrado = pd.concat([no_duplicados, duplicados_validos], ignore_index=True)

# Ver resultado
jugadores_total_filtrado


,player_id,player_name,team,team_id,competition,tackle_success_rate,tackles_successful,interception_success_rate,interceptions,clearances,...,height_cm,weight_kg,club_jersey_number,club_loaned_from,club_contract_valid_until,nationality_id,nationality_name,preferred_foot,release_clause_eur,posicion_mas_jugada
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,Spain - La Liga,56.86,29,62.75,32,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3063,Danilo Luiz da Silva,Real Madrid,220,Spain - La Liga,64.18,43,61.11,33,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3084,Christian Atsu Twasam,Málaga,223,Spain - La Liga,66.67,4,66.67,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3130,Gaël Kakuta,Sevilla,213,Spain - La Liga,100.00,2,0.00,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3160,Rene Krhin,Granada,1049,Spain - La Liga,64.71,22,71.05,27,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,7788,Ciro Immobile,Torino,241,Italy - Serie A,66.67,6,100.00,2,3,...,182.0,70.0,16.0,NaN,2020.0,27.0,Italy,Right,NaN,Left Center Forward
1936,7852,Tim Matavž,Genoa,233,Italy - Serie A,100.00,2,0.00,0,2,...,188.0,74.0,1.0,NaN,2019.0,27.0,Italy,Right,NaN,Left Center Forward
1937,8217,Kingsley Coman,Juventus,224,Italy - Serie A,0.00,0,0.00,0,0,...,179.0,73.0,20.0,NaN,2016.0,27.0,Italy,Right,NaN,Left Center Forward
1938,27853,Mauro Matías Zárate,Fiorentina,239,Italy - Serie A,87.50,7,80.00,4,1,...,188.0,80.0,18.0,NaN,2019.0,45.0,Spain,Right,NaN,Left Wing


In [10]:
# Filtrar jugadores que están cedidos (club_loaned_from no es NaN)
cedidos = jugadores_total_filtrado[jugadores_total_filtrado['club_loaned_from'].notna()]

# Mostrar algunas columnas relevantes
print(cedidos[['player_id', 'player_name', 'team', 'club_loaned_from']])


      player_id                    player_name             team  \
588        3312                   Demarai Gray   Leicester City   
621        3445   Radamel Falcao García Zárate          Chelsea   
717        3658                 DeAndre Yedlin       Sunderland   
756        3963                   Victor Moses  West Ham United   
779        4346  Gilbert Gianelli Imbula Wanga       Stoke City   
...         ...                            ...              ...   
1748       9404                 Luca Caldirola     Darmstadt 98   
1764      10435                 Łukasz Załuska     Darmstadt 98   
1770      10902                Philipp Hosiner          FC Köln   
1814      16502            László Kleinheisler    Werder Bremen   
1931       9461                     Timm Klose     Norwich City   

         club_loaned_from  
588          Swansea City  
621             AS Monaco  
717     Tottenham Hotspur  
756               Chelsea  
779               Chelsea  
...                   ...  

In [11]:
jugadores_total_filtrado["team"].unique()

array(['Real Sociedad', 'Real Madrid', 'Málaga', 'Sevilla', 'Granada',
       'Valencia', 'Villarreal', 'Barcelona', 'Real Betis', 'Las Palmas',
       'Athletic Club', 'Celta Vigo', 'Rayo Vallecano', 'Espanyol',
       'Atlético Madrid', 'RC Deportivo La Coruña', 'Levante UD', 'Eibar',
       'Sporting Gijón', 'Getafe', 'Chelsea', 'West Bromwich Albion',
       'Manchester United', 'Tottenham Hotspur', 'Aston Villa',
       'Newcastle United', 'AFC Bournemouth', 'Manchester City',
       'West Ham United', 'Crystal Palace', 'Southampton', 'Sunderland',
       'Everton', 'Swansea City', 'Watford', 'Stoke City',
       'Leicester City', 'Norwich City', 'Liverpool', 'Arsenal', 'Torino',
       'Lazio', 'Fiorentina', 'AS Roma', 'AC Milan', 'Empoli', 'Genoa',
       'Inter Milan', 'Juventus', 'Atalanta', 'Udinese', 'Napoli',
       'Sassuolo', 'Hellas Verona', 'Palermo', 'Sampdoria', 'Bologna',
       'Chievo', 'Carpi', 'Frosinone', 'Ingolstadt', 'Hannover 96',
       'Schalke 04', 'Bayer 

In [12]:
jugadores_total_filtrado["club_loaned_from"].unique()

array([nan, 'Swansea City', 'AS Monaco', 'Tottenham Hotspur', 'Chelsea',
       'Stade Rennais FC', 'Rubin Kazan', 'Liverpool', 'Fulham',
       'Arsenal', 'West Ham United', 'FC Barcelona', 'VfB Stuttgart',
       'Zenit St. Petersburg', 'Olympique de Marseille',
       'Atlético de Madrid', 'Galatasaray SK', 'Roma', 'Udinese Calcio',
       'Borussia Dortmund', 'Inter', 'Paris Saint-Germain',
       'U.S. Sassuolo Calcio', 'Palermo', 'Pescara', 'Napoli',
       'U.C. Sampdoria', 'Genoa', 'Juventus', 'Sunderland', 'Cagliari',
       'AC Milan', 'Grasshopper Club Zürich', 'FC Sion', 'Fenerbahçe SK',
       'Hellas Verona', 'Standard de Liège', 'Club Brugge KV',
       'Sevilla FC', 'Granada CF', 'Bologna', 'FC Bayern München',
       'Bayer 04 Leverkusen', 'FC Schalke 04', 'SV Werder Bremen'],
      dtype=object)